In [6]:
import torchio as tio
from pathlib import Path
from utils.data_utils import volume_processing

def process_subjects(paths, h, w, depth):
    subjects = []
    for subject_path in paths:
        subject = tio.Subject({"MRI": tio.ScalarImage(subject_path)})
        mri_data = subject["MRI"]
        processed_mri_data = volume_processing(mri_data, h, w, depth)
        processed_subject = tio.Subject({"MRI": processed_mri_data})
        subjects.append(processed_subject)
    return subjects

In [7]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn.init
from pathlib import Path
import torchio as tio
import pandas as pd
from models.CNN_3D import Unsupervised_Segmentation_Model, myloss3D_opt
from utils.data_utils import volume_processing

# Load configuration
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

device_ids = [config["device"]]
device = torch.device("cuda:%d" % device_ids[0] if torch.cuda.is_available() else "cpu")
print(device)

####### DATASET ########
df = pd.read_excel(config["path_to_excel"])

h, w, depth = config["h"], config["w"], config["depth"]

# T2 STIR
train_list_T2 = df.loc[((df["data_split"] == "TRAIN") & (df["modality"] == config["modality_2"]))]
train_list_T2 = train_list_T2["FILE"]
val_list_T2 = df.loc[((df["data_split"] == "VAL") & (df["modality"] == config["modality_2"]))]
val_list_T2 = val_list_T2["FILE"]
train_paths_T2 = [Path(i) for i in train_list_T2]
val_paths_T2 = [Path(i) for i in val_list_T2]

# T1 TSE
train_list_T1 = df.loc[((df["data_split"] == "TRAIN") & (df["modality"] == config["modality_1"]))]
train_list_T1 = train_list_T1["FILE"]
val_list_T1 = df.loc[((df["data_split"] == "VAL") & (df["modality"] == config["modality_1"]))]
val_list_T1 = val_list_T1["FILE"]
train_paths_T1 = [Path(i) for i in train_list_T1]
val_paths_T1 = [Path(i) for i in val_list_T1]

train_sbj_T2 = process_subjects(train_paths_T2, h, w, depth)
val_sbj_T2 = process_subjects(val_paths_T2, h, w, depth)

train_sbj_T1 = process_subjects(train_paths_T1, h, w, depth)


val_sbj_T1 = process_subjects(val_paths_T1, h, w, depth)



cuda:1


/home/annac/miniconda3/envs/pytorchenv/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/annac/miniconda3/envs/pytorchenv/lib/python3.7/site-packages/torchio/transforms/transform.py:158: RuntimeWarning: Output shape (97, 97, 10) != target shape (96, 96, 10). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/annac/miniconda3/envs/pytorchenv/lib/python3.7/site-packages/torchio/transforms/transform.py:158: RuntimeWarning: Output shape (97, 96, 10) != target shape (96, 96, 10). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/annac/miniconda3/envs/pytorchenv/lib/python3.7/site-packages/torchio/transforms/transform.py:158: RuntimeWarning: Output shape (96, 97, 10) != target shape (96, 96, 10). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


In [2]:
device_ids = [3]
device = torch.device("cuda:%d" % device_ids[0] if torch.cuda.is_available() else "cpu")
print(device)

cuda:3


### Dataset Creation

In [3]:
df = pd.read_excel('/home/annac/Documents/3D Bone Tumor Segmentation/Tables/modality_data_split801010.xlsx')

/home/annac/miniconda3/envs/pytorchenv/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
# T2 STIR
train_list_T2 = df.loc[((df['data_split'] == 'TRAIN') & (df['modality'] == 'T2'))]
train_list_T2 = train_list_T2['FILE']
val_list_T2 = df.loc[((df['data_split'] == 'VAL') & (df['modality'] == 'T2'))]
val_list_T2 = val_list_T2['FILE']
train_paths_T2 = [Path(i) for i in train_list_T2]
val_paths_T2 = [Path(i) for i in val_list_T2]

# T1 TSE
train_list_T1 = df.loc[((df['data_split'] == 'TRAIN') & (df['modality'] == 'T1_FS_KM'))]
train_list_T1 = train_list_T1['FILE']
val_list_T1 = df.loc[((df['data_split'] == 'VAL') & (df['modality'] == 'T1_FS_KM'))]
val_list_T1 = val_list_T1['FILE']
train_paths_T1 = [Path(i) for i in train_list_T1]
val_paths_T1 = [Path(i) for i in val_list_T1]

In [5]:
train_sbj_T2 = []
for subject_path in train_paths_T2:
    subject = tio.Subject({"MRI":tio.ScalarImage(subject_path)})
    mri_data = subject['MRI']
    processed_mri_data = volume_processing(mri_data, 500, 500, 20)
    processed_subject = tio.Subject({"MRI": processed_mri_data})
    train_sbj_T2.append(processed_subject)

val_sbj_T2 = []
for subject_path in val_paths_T2:
    subject = tio.Subject({"MRI":tio.ScalarImage(subject_path)})
    mri_data = subject['MRI']
    processed_mri_data = volume_processing(mri_data, 500, 500, 20)
    processed_subject = tio.Subject({"MRI": processed_mri_data})
    val_sbj_T2.append(processed_subject)

train_sbj_T1 = []
for subject_path in train_paths_T1:
    subject = tio.Subject({"MRI":tio.ScalarImage(subject_path)})
    mri_data = subject['MRI']
    processed_mri_data = volume_processing(mri_data, 500, 500, 20)
    processed_subject = tio.Subject({"MRI": processed_mri_data})
    train_sbj_T1.append(processed_subject)

val_sbj_T1 = []
for subject_path in val_paths_T1:
    subject = tio.Subject({"MRI":tio.ScalarImage(subject_path)})
    mri_data = subject['MRI']
    processed_mri_data = volume_processing(mri_data, 500, 500, 20)
    processed_subject = tio.Subject({"MRI": processed_mri_data})
    val_sbj_T1.append(processed_subject)

/home/annac/miniconda3/envs/pytorchenv/lib/python3.7/site-packages/torchio/transforms/transform.py:158: RuntimeWarning: Output shape (501, 501, 20) != target shape (500, 500, 20). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/annac/miniconda3/envs/pytorchenv/lib/python3.7/site-packages/torchio/transforms/transform.py:158: RuntimeWarning: Output shape (500, 501, 20) != target shape (500, 500, 20). Fixing with CropOrPad
  transformed = self.apply_transform(subject)
/home/annac/miniconda3/envs/pytorchenv/lib/python3.7/site-packages/torchio/transforms/transform.py:158: RuntimeWarning: Output shape (501, 500, 20) != target shape (500, 500, 20). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


In [3]:
train_transform = tio.Compose([tio.RandomFlip(axes=('LR',), flip_probability = 0.3),
    tio.RandomAffine(scales=(0.9, 1.2),degrees=15)])

In [4]:

train_dataset_T2 = tio.SubjectsDataset(train_sbj_T2, transform=train_transform)
val_dataset_T2 = tio.SubjectsDataset(val_sbj_T2) #, transform=val_transform)

print(f"There are {len(train_sbj_T2)} train subjects and {len(val_sbj_T2)} val subjects in T2 STIR")

train_dataset_T1 = tio.SubjectsDataset(train_sbj_T1, transform=train_transform)
val_dataset_T1 = tio.SubjectsDataset(val_sbj_T1) #, transform=val_transform)

print(f"There are {len(train_sbj_T1)} train subjects and {len(val_sbj_T1)} val subjects in T1 TSE")

There are 164 train subjects and 20 val subjects in T2 STIR
There are 164 train subjects and 20 val subjects in T1 TSE


In [5]:
batch_size = 4
num_workers = 4
train_loader_T2 = torch.utils.data.DataLoader(train_dataset_T2, batch_size=batch_size,
                                           num_workers=num_workers, shuffle=False,
                                              pin_memory=True)
val_loader_T2 = torch.utils.data.DataLoader(val_dataset_T2, batch_size=batch_size,
                                         num_workers=num_workers, shuffle=False,
                                            pin_memory=True)

train_loader_T1 = torch.utils.data.DataLoader(train_dataset_T1, batch_size=batch_size,
                                           num_workers=num_workers, shuffle=False,
                                              pin_memory=True)
val_loader_T1 = torch.utils.data.DataLoader(val_dataset_T1, batch_size=batch_size,
                                         num_workers=num_workers,shuffle=False,
                                            pin_memory=True)

### Train

In [ ]:
import warnings
warnings.simplefilter("ignore")
input_channels = 1
nConv = 2
nChannel = 30
lr = 0.001
num_epochs = 1000

model = Unsupervised_Segmentation_Model(input_channels, nConv=nConv, nChannel=nChannel)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)


mean_train_loss = []
mean_val_loss = []
best_val_loss = np.Inf
epochs_since_improvement = 0
patience = 10
converge = 0
for epoch in range(num_epochs):
    
    # TRAINING
    model.train()
    batch_idx=1
    epoch_train_loss = []
    epoch_val_loss = []
    for batch1, batch2 in zip(train_loader_T2, train_loader_T1):
        train_batch_T2 = batch1['MRI']['data'].to(device)
        train_batch_T1 = batch2['MRI']['data'].to(device)

        # Forward
        optimizer.zero_grad()
        pred_batch = model(train_batch_T2, train_batch_T1).to(device)
        torch.cuda.empty_cache()

        # Loss
        loss, nLabels = myloss3D_opt(pred_batch,train_batch_T2, nChannel*2, device)
        torch.cuda.empty_cache()

        # Backpropagation
        loss.backward()

        # Optimization step
        optimizer.step()

        epoch_train_loss.append(loss.item())
        print ('Epoch:',epoch,"| batch:", batch_idx,' | loss :', loss.item(), "| N Clusters: ", nLabels)
        batch_idx = batch_idx+1
        torch.cuda.empty_cache()


    mean_train_loss.append(np.mean(epoch_train_loss))

    # VALIDATION
    model.eval()
    with torch.no_grad():
        for batch1, batch2 in zip(val_loader_T2, val_loader_T1):
            val_batch_T2 = batch1['MRI']['data'].to(device)
            val_batch_T1 = batch2['MRI']['data'].to(device)

            # Forward
            pred_batch = model(val_batch_T2, val_batch_T1).to(device)

            # Loss
            loss, nLabels = myloss3D_opt(pred_batch,val_batch_T2, nChannel*2, device)

            epoch_val_loss.append(loss.item())
            del val_batch_T2, val_batch_T1, pred_batch, loss
            torch.cuda.empty_cache()

    mean_val_loss.append(np.mean(epoch_val_loss))
    print("Val Loss: ", np.mean(epoch_val_loss))

    # Check if the validation loss has improved
    if  np.mean(epoch_val_loss) < best_val_loss:
        best_val_loss = np.mean(epoch_val_loss)
        epochs_since_improvement = 0
        converge = epoch
        torch.save(model.state_dict(), '3D_MultiModal_060923_fliprot500.pt')
    else:
        epochs_since_improvement += 1

    # Check if training should be stopped
    if epochs_since_improvement >= patience:
        print('Training stopped because validation loss did not improve for {} epochs.'.format(patience))
        np.save('mean_train_loss_060923_fliprot500.npy', mean_train_loss)
        np.save('mean_val_loss_060923_fliprot500.npy', mean_val_loss)
        break

    if epoch == (num_epochs-1):
        np.save('mean_train_loss_060923_e_fliprot500.npy', mean_train_loss)
        np.save('mean_val_loss_060923_e_fliprot500.npy', mean_val_loss)
        break

